In [1]:
import pymc as pm
import numpy as np

from aesara.compile.builders import OpFromGraph

In [4]:
with pm.Model() as model:
    mu = pm.Normal("mu", [-5, 5], 4.)
    mix = pm.NormalMixture("mix", w=[0.5, 0.5], mu=mu, sigma=1.)
    
    prior = pm.sample_prior_predictive()

[RandomGeneratorSharedVariable(<Generator(PCG64) at 0x170DAB900>), TensorConstant{(2,) of 0.5}, normal_rv{0, (0, 0), floatX, False}.out]
[MarginalMixtureRV{inline=False}.0, mix]
[RandomGeneratorSharedVariable(<Generator(PCG64) at 0x170DAB900>), TensorConstant{(2,) of 0.5}, normal_rv{0, (0, 0), floatX, False}.out]
[MarginalMixtureRV{inline=False}.0, mix]


In [ ]:
class _LatentDirichletProcessRV(OpFromGraph):
    
    def update(self, node):
        return {node.inputs[0]: node.outputs[0]}
    


In [ ]:
class CustomNormal(pm.Normal):
    rv_op = pm.Normal.rv_op
    
    def __new__(cls, name, mu, sigma, observed=None, **kwargs):
        if observed is None:
            return super().__new__(cls, name=name, mu=mu, sigma=sigma, observed=None, **kwargs)
        else:
            return super().__new__(cls, name=name, mu=mu+3, sigma=sigma, observed=None, **kwargs)
        
    @classmethod
    def ndim_supp(cls):
        return 1

In [ ]:
with pm.Model() as model:
    norm = CustomNormal("norm", 0., 5., observed=np.array([1, 2, 3]))

In [ ]:
model.free_RVs